# 循环神经网络 Recurrent Neural Networks

## 1. 为什么要使用序列模型 Why sequence models

![Examples of sequence data.png](img/Examples of sequence data.png)

## 2. 标记符号 Notations

以命名实体识别（Named Entity Recognition）问题为例，我们需要一套标记符号，来描述这个监督学习问题。这里，用符号 $x^{(i)<t>}$ 表示第 i 个训练集的第 t 个词，用符号 $T_x^{(i)}$ 表示第 i 个训练集总共的单词数。目标变量 y 也类似。

**标记**:
- 上标 $[l]$ 表示第 $l^{th}$ 层对应的对象
    - 举例: $a^{[4]}$ 是第 $4^{th}$ 层激活层。 $W^{[5]}$ 和 $b^{[5]}$ 是第 $5^{th}$ 层的参数

- 上标 $(i)$ 表示第 $i^{th}$ 个训练样本相关的对象
    - 举例: $x^{(i)}$ 表示输入的第 $i^{th}$ 个训练样本

- 上标 $\langle t \rangle$ 表示在第 $t^{th}$ 个时间步长的对象 
    - 举例: $x^{\langle t \rangle}$ 表示输入x在第 $t^{th}$ 个时间步长的值。 $x^{(i)\langle t \rangle}$ 是训练样本 i 在第 $t^{th}$ 个时间步长的值
    
- 下标 $i$ 表示向量的第 $i^{th}$ 个元素
    - 举例: $a^{[l]}_i$ 表示激活层 $l$ 的第 $i^{th}$ 个元素

![Motivating example RNN.png](img/Motivating example RNN.png)

由于这里是一个NLP（Natural Language Processing）问题，就涉及到如何来表示单词。通常我们会构造一个词典，以30000~50000的单词量为最常见，也有100000级别的词典，甚至在一些互联网公司，百万级别的词典也不是没有。有了词典之后，每个单词就会有一个序号。而将所有单词进行One-Hot编码，就可以获得表示这个单词的向量。

![Representing words.png](img/Representing words.png)

## 3. 循环神经网络模型 Recurrent Neural Network Model

使用标准神经网络模型来处理序列数据，主要会面临以下两个问题：1）序列数据的输入输出长度，对于不同的样本，可以是不同的；2）标准神经网络不会共享对序列数据的位置相关特征。

![Why not a standard network.png](img/Why not a standard network.png)

循环神经网络具有“记忆”，因而对于自然语言处理和其它序列模型都非常有效。模型可以每次一个接受 $x^{<t>}$ （比如一个单词），通过隐藏层的激活情况，记住该项的信息/上下文，然后传递给下一个项目。

循环神经网络的模型：初始化 $a^{<0>}$ 为0向量，序列中每个单词的激活结果，作为下一个单词模型的输入之一。注意到这个RNN模型实际上是单向RNN，顺序在前的单词可以对顺序在后的单词施加影响，反之则不行。后面会说，经过一点简单的修改，也可以获得**双向RNN（Bidirected RNN）**

![Recurrent Neural Networks.png](img/RNN.png)

循环神经网络可以认为是上面单个循环神经网络单元的不断重复，每个循环神经网络单元的内部操作如下：

![rnn_step_forward](img/rnn_step_forward.png)

In [1]:
import numpy as np
from rnn_utils import *

In [2]:
# GRADED FUNCTION: rnn_cell_forward

def rnn_cell_forward(xt, a_prev, parameters):
    """
    Implements a single forward step of the RNN-cell as described in Figure (2)

    Arguments:
    xt -- your input data at timestep "t", numpy array of shape (n_x, m).
    a_prev -- Hidden state at timestep "t-1", numpy array of shape (n_a, m)
    parameters -- python dictionary containing:
                        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
                        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
                        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        ba --  Bias, numpy array of shape (n_a, 1)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
    Returns:
    a_next -- next hidden state, of shape (n_a, m)
    yt_pred -- prediction at timestep "t", numpy array of shape (n_y, m)
    cache -- tuple of values needed for the backward pass, contains (a_next, a_prev, xt, parameters)
    """
    
    # Retrieve parameters from "parameters"
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Wya = parameters["Wya"]
    ba = parameters["ba"]
    by = parameters["by"]
    
    ### START CODE HERE ### (≈2 lines)
    # compute next activation state using the formula given above
    a_next = np.tanh(Waa.dot(a_prev) + Wax.dot(xt) + ba)
    # compute output of the current cell using the formula given above
    yt_pred = softmax(Wya.dot(a_next) + by)
    ### END CODE HERE ###
    
    # store values you need for backward propagation in cache
    cache = (a_next, a_prev, xt, parameters)
    
    return a_next, yt_pred, cache

In [3]:
np.random.seed(1)
xt = np.random.randn(3,10)
a_prev = np.random.randn(5,10)
Waa = np.random.randn(5,5)
Wax = np.random.randn(5,3)
Wya = np.random.randn(2,5)
ba = np.random.randn(5,1)
by = np.random.randn(2,1)
parameters = {"Waa": Waa, "Wax": Wax, "Wya": Wya, "ba": ba, "by": by}

a_next, yt_pred, cache = rnn_cell_forward(xt, a_prev, parameters)
print("a_next[4] = ", a_next[4])
print("a_next.shape = ", a_next.shape)
print("yt_pred[1] =", yt_pred[1])
print("yt_pred.shape = ", yt_pred.shape)

a_next[4] =  [ 0.59584544  0.18141802  0.61311866  0.99808218  0.85016201  0.99980978
 -0.18887155  0.99815551  0.6531151   0.82872037]
a_next.shape =  (5, 10)
yt_pred[1] = [ 0.9888161   0.01682021  0.21140899  0.36817467  0.98988387  0.88945212
  0.36920224  0.9966312   0.9982559   0.17746526]
yt_pred.shape =  (2, 10)


**预期输出**: 

<table>
    <tr>
        <td>
            **a_next[4]**:
        </td>
        <td>
           [ 0.59584544  0.18141802  0.61311866  0.99808218  0.85016201  0.99980978
 -0.18887155  0.99815551  0.6531151   0.82872037]
        </td>
    </tr>
        <tr>
        <td>
            **a_next.shape**:
        </td>
        <td>
           (5, 10)
        </td>
    </tr>
        <tr>
        <td>
            **yt[1]**:
        </td>
        <td>
           [ 0.9888161   0.01682021  0.21140899  0.36817467  0.98988387  0.88945212
  0.36920224  0.9966312   0.9982559   0.17746526]
        </td>
    </tr>
        <tr>
        <td>
            **yt.shape**:
        </td>
        <td>
           (2, 10)
        </td>
    </tr>

</table>

RNN的前向传播过程，就是对RNN单元的重复。加入输入序列包含10个时间步长，那么对RNN单元复制10次即可。每个单元的输入是前一个单元隐藏层的激活值($a^{\langle t-1 \rangle}$)以及当前时间步长的输入数据($x^{\langle t \rangle}$)。而对应的输出是当前单元隐藏状态($a^{\langle t \rangle}$)和当前时间步长的与预测值($y^{\langle t \rangle}$)。

<img src="img/rnn.png" style="width:800px;height:300px;">

In [4]:
# GRADED FUNCTION: rnn_forward

def rnn_forward(x, a0, parameters):
    """
    Implement the forward propagation of the recurrent neural network described in Figure (3).

    Arguments:
    x -- Input data for every time-step, of shape (n_x, m, T_x).
    a0 -- Initial hidden state, of shape (n_a, m)
    parameters -- python dictionary containing:
                        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
                        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
                        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        ba --  Bias numpy array of shape (n_a, 1)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)

    Returns:
    a -- Hidden states for every time-step, numpy array of shape (n_a, m, T_x)
    y_pred -- Predictions for every time-step, numpy array of shape (n_y, m, T_x)
    caches -- tuple of values needed for the backward pass, contains (list of caches, x)
    """
    
    # Initialize "caches" which will contain the list of all caches
    caches = []
    
    # Retrieve dimensions from shapes of x and parameters["Wya"]
    n_x, m, T_x = x.shape
    n_y, n_a = parameters["Wya"].shape
    
    ### START CODE HERE ###
    
    # initialize "a" and "y" with zeros (≈2 lines)
    a = np.zeros((n_a, m, T_x))
    y_pred = np.zeros((n_y, m, T_x))
    
    # Initialize a_next (≈1 line)
    a_next = a0
    
    # loop over all time-steps
    for t in range(T_x):
        # Update next hidden state, compute the prediction, get the cache (≈1 line)
        a_next, yt_pred, cache = rnn_cell_forward(x[:,:,t], a_next, parameters)
        # Save the value of the new "next" hidden state in a (≈1 line)
        a[:,:,t] = a_next
        # Save the value of the prediction in y (≈1 line)
        y_pred[:,:,t] = yt_pred
        # Append "cache" to "caches" (≈1 line)
        caches.append(cache)
        
    ### END CODE HERE ###
    
    # store values needed for backward propagation in cache
    caches = (caches, x)
    
    return a, y_pred, caches

In [5]:
np.random.seed(1)
x = np.random.randn(3,10,4)
a0 = np.random.randn(5,10)
Waa = np.random.randn(5,5)
Wax = np.random.randn(5,3)
Wya = np.random.randn(2,5)
ba = np.random.randn(5,1)
by = np.random.randn(2,1)
parameters = {"Waa": Waa, "Wax": Wax, "Wya": Wya, "ba": ba, "by": by}

a, y_pred, caches = rnn_forward(x, a0, parameters)
print("a[4][1] = ", a[4][1])
print("a.shape = ", a.shape)
print("y_pred[1][3] =", y_pred[1][3])
print("y_pred.shape = ", y_pred.shape)
print("caches[1][1][3] =", caches[1][1][3])
print("len(caches) = ", len(caches))

a[4][1] =  [-0.99999375  0.77911235 -0.99861469 -0.99833267]
a.shape =  (5, 10, 4)
y_pred[1][3] = [ 0.79560373  0.86224861  0.11118257  0.81515947]
y_pred.shape =  (2, 10, 4)
caches[1][1][3] = [-1.1425182  -0.34934272 -0.20889423  0.58662319]
len(caches) =  2


**预期输出**:

<table>
    <tr>
        <td>
            **a[4][1]**:
        </td>
        <td>
           [-0.99999375  0.77911235 -0.99861469 -0.99833267]
        </td>
    </tr>
        <tr>
        <td>
            **a.shape**:
        </td>
        <td>
           (5, 10, 4)
        </td>
    </tr>
        <tr>
        <td>
            **y[1][3]**:
        </td>
        <td>
           [ 0.79560373  0.86224861  0.11118257  0.81515947]
        </td>
    </tr>
        <tr>
        <td>
            **y.shape**:
        </td>
        <td>
           (2, 10, 4)
        </td>
    </tr>
        <tr>
        <td>
            **cache[1][1][3]**:
        </td>
        <td>
           [-1.1425182  -0.34934272 -0.20889423  0.58662319]
        </td>
    </tr>
        <tr>
        <td>
            **len(cache)**:
        </td>
        <td>
           2
        </td>
    </tr>

</table>

部分参数可以合并，从而简化计算：

![Simplified RNN notation.png](img/Simplified RNN notation.png)

## 4. 通过时间的反向传播 Backpropagation through time

![Backpropagation through time.png](img/Backpropagation through time.png)

## 5. 不同类型的循环神经网络 Different types of RNNs

上面提到的循环神经网络，都属于 $T_x = T_y$ 的多对多的循环神经网络，适用于命名实体识别问题。另外，也存在着其它不同类型的循环神经网络。

比如对于情感分析问题，可以构造多对一的RNN，甚至一对一，输出可以是二分类，也可以是1星到5星的多分类；而对于音乐生产问题，则可以构造一对多的RNN，其中输入可能是音乐类型，输出是音乐的序列数据；对于机器翻译问题，可以构造另一种输入输出的长度不同的RNN。

![Summary of RNN types.png](img/Summary of RNN types.png)

## 6. 语言模型和序列生成 Language Model and sequence generation

语言模型是自然语言处理的一个基础性而重要的任务，而RNN可以有效地进行语言模型建模。

以语音识别为例，下面两个句子的读音是完全一致的，语言模型可以给出每个句子的概率，通过这个概率帮助语音识别系统判断应该输出哪个句子。语言模型也可以用来帮助机器翻译。

![What is language modelling.png](img/What is language modelling.png)

建立语言模型，首先需要有一个**语料库 corpus**作为训练集，所谓语料库就是特定语言的一大批句子。第二部是**Tokenize**，也就是建立词典，并将语料库中的单词转换为向量的过程。注意到一般词典中会包含两个特殊的词，< EOS >表示句子结束，而< UNK >表示未知的词。

![Language modelling with an RNN.png](img/Language modelling with an RNN.png)

RNN实现的语言模型，令 $x^{<t>}=y^{<t-1>}$。$\hat{y}^{<1>}$ 输出的是第一个词的概率分布，是一个对字典中所有词的softmax回归。而 $\hat{y}^{<2>}$ 输出的是给定第一个词时，第二个词取值的条件概率分布。以此类推。

而上面提到的句子的概率，就是所有词概率的乘积，如右下角所示。

![RNN model.png](img/RNN model.png)

## 7. 生成新序列 Sampling novel sequences

有了训练完成的语言模型后，我们可以根据这个模型给定的多项分布以及后续的条件概率多项分布，来生成新的句子。如果词典中包含了 < EOS > 字符，则可以在生成该字符后中止句子。而对于 < UNK > 字符，如果不希望生成的句子里有未知字符，则拒绝该字符，继续生成即可。

![Sampling a sequence from a trained RNN.png](img/Sampling a sequence from a trained RNN.png)

上面介绍的语言模型，是单词级别的。也可以构造字符级别的语言模型。字符级别的语言模型，就不需要额外处理未知的单词了，但其计算量更大，对于生成长序列时表现也不够好，目前只有在少数情况下会使用字符级别的语言模型。

![Character-level language model.png](img/Character-level language model.png)

## 8. 循环神经网络的梯度消失问题 Vanishing gradients with RNNs

下面列举了两个句子，根据主语的单复数（cat or cats），跟在一大段定语从句后面的be动词会需要不同（was or were）。而目前提到的RNN，处理这个**长依赖 long-range dependencies**的能力不强，究其原因，可以归结于梯度消失。RNN中每个单词，都相当于神经网络的一层。当RNN的网络非常深时，很难使得反向传播比较强地影响到最初几层。所以基础的RNN结构，仅在输出 $y^{\langle t \rangle}$ 主要依赖于局部上下文 (即局部信息 $x^{\langle t' \rangle}$ ，其中 $t'$ 离 $t$ 很近) 的情况下表现良好。

为了应对RNN的梯度消失问题，有一系列的解决方案。以GRU和LSTM为主要代表。

RNN的梯度爆炸：只需要进行**梯度裁剪gradient cliping**即可，相对好解决。

![Vanishing gradients with RNNs.png](img/Vanishing gradients with RNNs.png)

## 9. GRU单元 Gated Recurrent Unit

![RNN unit.png](img/RNN unit.png)

![GRU simplified.png](img/GRU simplified.png)

![Full GRU.png](img/Full GRU.png)

## 10. 长短期记忆单元 Long Short Term Memory

LSTM是更复杂的模型，使用也更为广泛。GRU的计算量会小一些。LSTM单元结构如下：

<img src="img/LSTM.png" style="width:500;height:400px;">
<caption><center> **Figure 4**: LSTM-cell. This tracks and updates a "cell state" or memory variable $c^{\langle t \rangle}$ at every time-step, which can be different from $a^{\langle t \rangle}$. </center></caption>

#### - 遗忘门 Forget gate

假设我们在阅读文章，希望用LSTM来记录语法结构，比如记录下主语是单数还是复数。如果主语从单数变成了复数，我们需要有某种机制可以忘记之前保存的关于单复数状态的记忆值。LSTM中的遗忘门用于达成这个目的：

$$\Gamma_f^{\langle t \rangle} = \sigma(W_f[a^{\langle t-1 \rangle}, x^{\langle t \rangle}] + b_f)\tag{1} $$

这里，权重 $W_f$ 控制遗忘门的行为。我们将 $[a^{\langle t-1 \rangle} 和 x^{\langle t \rangle}]$ 合并，用 $W_f$ 来乘。这个等式的结果是一个值在0和1之间的向量 $\Gamma_f^{\langle t \rangle}$。而遗忘门向量最终会和之前单元的状态变量 $c^{\langle t-1 \rangle}$ 进行元素级别的相乘。所以如果 $\Gamma_f^{\langle t \rangle}$ 的值是0（或者接近0），LSTM就会在对应的 $c^{\langle t-1 \rangle}$ 中移除相关的信息（比如：之前是单数的主语）。而如果值是1，则会继续保留信息。

#### - 更新门 Update gate

一旦我们遗忘了主语是单数，接下来就需要一种机制来反应新的主语是复数。下面是更新门的公式：

$$\Gamma_u^{\langle t \rangle} = \sigma(W_u[a^{\langle t-1 \rangle}, x^{\{t\}}] + b_u)\tag{2} $$ 

和遗忘门类似，这里 $\Gamma_u^{\langle t \rangle}$ 也是一个值在0到1之间的向量。它会和 $\tilde{c}^{\langle t \rangle}$ 进行元素级别的相乘来计算 $c^{\langle t \rangle}$。

#### - 更新单元 Updating the cell 

要更新主语，我们需要创建一个新的向量，使其可以和之前的单元状态变量相加，这里的公式为：

$$ \tilde{c}^{\langle t \rangle} = \tanh(W_c[a^{\langle t-1 \rangle}, x^{\langle t \rangle}] + b_c)\tag{3} $$

最终，新的单元状态变量为：

$$ c^{\langle t \rangle} = \Gamma_f^{\langle t \rangle}* c^{\langle t-1 \rangle} + \Gamma_u^{\langle t \rangle} *\tilde{c}^{\langle t \rangle} \tag{4} $$


#### - 输出门 Output gate

公式如下：

$$ \Gamma_o^{\langle t \rangle}=  \sigma(W_o[a^{\langle t-1 \rangle}, x^{\langle t \rangle}] + b_o)\tag{5}$$ 
$$ a^{\langle t \rangle} = \Gamma_o^{\langle t \rangle}* \tanh(c^{\langle t \rangle})\tag{6} $$

在公式5中，我们会使用sigmoid函数。而在公式6中，对之前的状态使用tanh函数。

In [6]:
# GRADED FUNCTION: lstm_cell_forward

def lstm_cell_forward(xt, a_prev, c_prev, parameters):
    """
    Implement a single forward step of the LSTM-cell as described in Figure (4)

    Arguments:
    xt -- your input data at timestep "t", numpy array of shape (n_x, m).
    a_prev -- Hidden state at timestep "t-1", numpy array of shape (n_a, m)
    c_prev -- Memory state at timestep "t-1", numpy array of shape (n_a, m)
    parameters -- python dictionary containing:
                        Wf -- Weight matrix of the forget gate, numpy array of shape (n_a, n_a + n_x)
                        bf -- Bias of the forget gate, numpy array of shape (n_a, 1)
                        Wi -- Weight matrix of the update gate, numpy array of shape (n_a, n_a + n_x)
                        bi -- Bias of the update gate, numpy array of shape (n_a, 1)
                        Wc -- Weight matrix of the first "tanh", numpy array of shape (n_a, n_a + n_x)
                        bc --  Bias of the first "tanh", numpy array of shape (n_a, 1)
                        Wo -- Weight matrix of the output gate, numpy array of shape (n_a, n_a + n_x)
                        bo --  Bias of the output gate, numpy array of shape (n_a, 1)
                        Wy -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
                        
    Returns:
    a_next -- next hidden state, of shape (n_a, m)
    c_next -- next memory state, of shape (n_a, m)
    yt_pred -- prediction at timestep "t", numpy array of shape (n_y, m)
    cache -- tuple of values needed for the backward pass, contains (a_next, c_next, a_prev, c_prev, xt, parameters)
    
    Note: ft/it/ot stand for the forget/update/output gates, cct stands for the candidate value (c tilde),
          c stands for the memory value
    """

    # Retrieve parameters from "parameters"
    Wf = parameters["Wf"]
    bf = parameters["bf"]
    Wi = parameters["Wi"]
    bi = parameters["bi"]
    Wc = parameters["Wc"]
    bc = parameters["bc"]
    Wo = parameters["Wo"]
    bo = parameters["bo"]
    Wy = parameters["Wy"]
    by = parameters["by"]
    
    # Retrieve dimensions from shapes of xt and Wy
    n_x, m = xt.shape
    n_y, n_a = Wy.shape

    ### START CODE HERE ###
    # Concatenate a_prev and xt (≈3 lines)
    concat = np.zeros((n_a + n_x, m))
    concat[: n_a, :] = a_prev
    concat[n_a :, :] = xt

    # Compute values for ft, it, cct, c_next, ot, a_next using the formulas given figure (4) (≈6 lines)
    ft = sigmoid(Wf.dot(concat) + bf)
    it = sigmoid(Wi.dot(concat) + bi)
    cct = np.tanh(Wc.dot(concat) + bc)
    c_next = it * cct + ft * c_prev
    ot = sigmoid(Wo.dot(concat) + bo)
    a_next = ot * np.tanh(c_next)
    
    # Compute prediction of the LSTM cell (≈1 line)
    yt_pred = softmax(Wy.dot(a_next) + by)
    ### END CODE HERE ###

    # store values needed for backward propagation in cache
    cache = (a_next, c_next, a_prev, c_prev, ft, it, cct, ot, xt, parameters)

    return a_next, c_next, yt_pred, cache

In [7]:
np.random.seed(1)
xt = np.random.randn(3,10)
a_prev = np.random.randn(5,10)
c_prev = np.random.randn(5,10)
Wf = np.random.randn(5, 5+3)
bf = np.random.randn(5,1)
Wi = np.random.randn(5, 5+3)
bi = np.random.randn(5,1)
Wo = np.random.randn(5, 5+3)
bo = np.random.randn(5,1)
Wc = np.random.randn(5, 5+3)
bc = np.random.randn(5,1)
Wy = np.random.randn(2,5)
by = np.random.randn(2,1)

parameters = {"Wf": Wf, "Wi": Wi, "Wo": Wo, "Wc": Wc, "Wy": Wy, "bf": bf, "bi": bi, "bo": bo, "bc": bc, "by": by}

a_next, c_next, yt, cache = lstm_cell_forward(xt, a_prev, c_prev, parameters)
print("a_next[4] = ", a_next[4])
print("a_next.shape = ", c_next.shape)
print("c_next[2] = ", c_next[2])
print("c_next.shape = ", c_next.shape)
print("yt[1] =", yt[1])
print("yt.shape = ", yt.shape)
print("cache[1][3] =", cache[1][3])
print("len(cache) = ", len(cache))

a_next[4] =  [-0.66408471  0.0036921   0.02088357  0.22834167 -0.85575339  0.00138482
  0.76566531  0.34631421 -0.00215674  0.43827275]
a_next.shape =  (5, 10)
c_next[2] =  [ 0.63267805  1.00570849  0.35504474  0.20690913 -1.64566718  0.11832942
  0.76449811 -0.0981561  -0.74348425 -0.26810932]
c_next.shape =  (5, 10)
yt[1] = [ 0.79913913  0.15986619  0.22412122  0.15606108  0.97057211  0.31146381
  0.00943007  0.12666353  0.39380172  0.07828381]
yt.shape =  (2, 10)
cache[1][3] = [-0.16263996  1.03729328  0.72938082 -0.54101719  0.02752074 -0.30821874
  0.07651101 -1.03752894  1.41219977 -0.37647422]
len(cache) =  10


**预期输出**:

<table>
    <tr>
        <td>
            **a_next[4]**:
        </td>
        <td>
           [-0.66408471  0.0036921   0.02088357  0.22834167 -0.85575339  0.00138482
  0.76566531  0.34631421 -0.00215674  0.43827275]
        </td>
    </tr>
        <tr>
        <td>
            **a_next.shape**:
        </td>
        <td>
           (5, 10)
        </td>
    </tr>
        <tr>
        <td>
            **c_next[2]**:
        </td>
        <td>
           [ 0.63267805  1.00570849  0.35504474  0.20690913 -1.64566718  0.11832942
  0.76449811 -0.0981561  -0.74348425 -0.26810932]
        </td>
    </tr>
        <tr>
        <td>
            **c_next.shape**:
        </td>
        <td>
           (5, 10)
        </td>
    </tr>
        <tr>
        <td>
            **yt[1]**:
        </td>
        <td>
           [ 0.79913913  0.15986619  0.22412122  0.15606108  0.97057211  0.31146381
  0.00943007  0.12666353  0.39380172  0.07828381]
        </td>
    </tr>
        <tr>
        <td>
            **yt.shape**:
        </td>
        <td>
           (2, 10)
        </td>
    </tr>
    <tr>
        <td>
            **cache[1][3]**:
        </td>
        <td>
           [-0.16263996  1.03729328  0.72938082 -0.54101719  0.02752074 -0.30821874
  0.07651101 -1.03752894  1.41219977 -0.37647422]
        </td>
    </tr>
        <tr>
        <td>
            **len(cache)**:
        </td>
        <td>
           10
        </td>
    </tr>

</table>

同样地，LSTM架构的RNN，就是不断重复LSTM单元。

<img src="img/LSTM_rnn.png" style="width:500;height:300px;">
<caption><center> **Figure 4**: LSTM over multiple time-steps. </center></caption>

In [8]:
# GRADED FUNCTION: lstm_forward

def lstm_forward(x, a0, parameters):
    """
    Implement the forward propagation of the recurrent neural network using an LSTM-cell described in Figure (3).

    Arguments:
    x -- Input data for every time-step, of shape (n_x, m, T_x).
    a0 -- Initial hidden state, of shape (n_a, m)
    parameters -- python dictionary containing:
                        Wf -- Weight matrix of the forget gate, numpy array of shape (n_a, n_a + n_x)
                        bf -- Bias of the forget gate, numpy array of shape (n_a, 1)
                        Wi -- Weight matrix of the update gate, numpy array of shape (n_a, n_a + n_x)
                        bi -- Bias of the update gate, numpy array of shape (n_a, 1)
                        Wc -- Weight matrix of the first "tanh", numpy array of shape (n_a, n_a + n_x)
                        bc -- Bias of the first "tanh", numpy array of shape (n_a, 1)
                        Wo -- Weight matrix of the output gate, numpy array of shape (n_a, n_a + n_x)
                        bo -- Bias of the output gate, numpy array of shape (n_a, 1)
                        Wy -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
                        
    Returns:
    a -- Hidden states for every time-step, numpy array of shape (n_a, m, T_x)
    y -- Predictions for every time-step, numpy array of shape (n_y, m, T_x)
    caches -- tuple of values needed for the backward pass, contains (list of all the caches, x)
    """

    # Initialize "caches", which will track the list of all the caches
    caches = []
    
    ### START CODE HERE ###
    # Retrieve dimensions from shapes of x and parameters['Wy'] (≈2 lines)
    n_x, m, T_x = x.shape
    n_y, n_a = parameters["Wy"].shape
    
    # initialize "a", "c" and "y" with zeros (≈3 lines)
    a = np.zeros((n_a, m, T_x))
    c = np.zeros((n_a, m, T_x))
    y = np.zeros((n_y, m, T_x))
    
    # Initialize a_next and c_next (≈2 lines)
    a_next = a0
    c_next = np.zeros((n_a, m))
    
    # loop over all time-steps
    for t in range(T_x):
        # Update next hidden state, next memory state, compute the prediction, get the cache (≈1 line)
        a_next, c_next, yt, cache = lstm_cell_forward(x[:,:,t], a_next, c_next, parameters)
        # Save the value of the new "next" hidden state in a (≈1 line)
        a[:,:,t] = a_next
        # Save the value of the prediction in y (≈1 line)
        y[:,:,t] = yt
        # Save the value of the next cell state (≈1 line)
        c[:,:,t]  = c_next
        # Append the cache into caches (≈1 line)
        caches.append(cache)
        
    ### END CODE HERE ###
    
    # store values needed for backward propagation in cache
    caches = (caches, x)

    return a, y, c, caches

In [9]:
np.random.seed(1)
x = np.random.randn(3,10,7)
a0 = np.random.randn(5,10)
Wf = np.random.randn(5, 5+3)
bf = np.random.randn(5,1)
Wi = np.random.randn(5, 5+3)
bi = np.random.randn(5,1)
Wo = np.random.randn(5, 5+3)
bo = np.random.randn(5,1)
Wc = np.random.randn(5, 5+3)
bc = np.random.randn(5,1)
Wy = np.random.randn(2,5)
by = np.random.randn(2,1)

parameters = {"Wf": Wf, "Wi": Wi, "Wo": Wo, "Wc": Wc, "Wy": Wy, "bf": bf, "bi": bi, "bo": bo, "bc": bc, "by": by}

a, y, c, caches = lstm_forward(x, a0, parameters)
print("a[4][3][6] = ", a[4][3][6])
print("a.shape = ", a.shape)
print("y[1][4][3] =", y[1][4][3])
print("y.shape = ", y.shape)
print("caches[1][1[1]] =", caches[1][1][1])
print("c[1][2][1]", c[1][2][1])
print("len(caches) = ", len(caches))

a[4][3][6] =  0.172117767533
a.shape =  (5, 10, 7)
y[1][4][3] = 0.95087346185
y.shape =  (2, 10, 7)
caches[1][1[1]] = [ 0.82797464  0.23009474  0.76201118 -0.22232814 -0.20075807  0.18656139
  0.41005165]
c[1][2][1] -0.855544916718
len(caches) =  2


**预期输出**:

<table>
    <tr>
        <td>
            **a[4][3][6]** =
        </td>
        <td>
           0.172117767533
        </td>
    </tr>
        <tr>
        <td>
            **a.shape** =
        </td>
        <td>
           (5, 10, 7)
        </td>
    </tr>
        <tr>
        <td>
            **y[1][4][3]** =
        </td>
        <td>
           0.95087346185
        </td>
    </tr>
        <tr>
        <td>
            **y.shape** =
        </td>
        <td>
           (2, 10, 7)
        </td>
    </tr>
        <tr>
        <td>
            **caches[1][1][1]** =
        </td>
        <td>
           [ 0.82797464  0.23009474  0.76201118 -0.22232814 -0.20075807  0.18656139
  0.41005165]
        </td>
        
     </tr>
        <tr>
        <td>
            **c[1][2][1]** =
        </td>
        <td>
           -0.855544916718
        </td>
    </tr>       
        
    </tr>
        <tr>
        <td>
            **len(caches)** =
        </td>
        <td>
           2
        </td>
    </tr>

</table>

## 11. 双向循环神经网络 Bidirectional RNN

BRNN和LSTM几乎是NLP问题中的标配。

缺点：需要完整的序列数据来预测结果，而无法仅使用截止到当前的序列数据。对于实时要求高的系统，复杂度更高。

![Bidirectional RNN.png](img/Bidirectional RNN.png)

## 12. 深度循环神经网络 Deep RNNs

略。